In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import evaluate

# Load the movielens-100k dataset (download it if needed),
# and split it into 3 folds for cross-validation.
data = Dataset.load_builtin('ml-100k')
data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = SVD()

# Evaluate performances of our algorithm on the dataset.
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])

print(perf)

ContextualVersionConflict: (numpy 1.11.1 (/home/kiki/anaconda3/lib/python3.5/site-packages), Requirement.parse('numpy>=1.11.2'), {'surprise'})